In [ ]:
import plotly.graph_objects as go
import vtk
import numpy as np

# 读取 VTK 文件
reader = vtk.vtkPolyDataReader()
reader.SetFileName('output_pynq.vtk')
reader.Update()
polydata = reader.GetOutput()

# 提取数据
points = polydata.GetPoints()
vertices = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])

triangles = []
for i in range(polydata.GetNumberOfCells()):
    cell = polydata.GetCell(i)
    if cell.GetNumberOfPoints() == 3:
        triangles.append([cell.GetPointId(j) for j in range(3)])
triangles = np.array(triangles)

# 翻转 Z 轴
vertices[:, 2] = -vertices[:, 2]

# 计算每个顶点的 Z 值用于颜色映射(可选，让表面有深度感)
intensity = vertices[:, 2]

# 创建 mesh3d - 医学图像优化版本
fig = go.Figure(data=[
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=triangles[:, 0],
        j=triangles[:, 1],
        k=triangles[:, 2],
        
        # 方案1: 使用单一颜色（类似骨骼的米白色）
        #color='#F5E6D3',  # 骨白色
        
        # 方案2: 使用渐变色显示深度（取消注释下面3行，注释上面1行）
         intensity=intensity,
         colorscale='YlOrRd',  # 暖色调，适合医学图像
         colorbar=dict(title="深度"),
        
        opacity=1.0,
        
        # 优化光照效果
        
        
        lightposition=dict(
            x=100,
            y=200,
            z=300
        ),
        
        # 显示边缘（可选）
        # flatshading=True,  # 平面着色，更清晰
    )
])

# 优化布局
fig.update_layout(
    scene=dict(
        aspectmode='data',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5),  # 相机位置
            center=dict(x=0, y=0, z=0),
            up=dict(x=0, y=0, z=1)          # Z轴向上
        ),
        xaxis=dict(
            backgroundcolor="rgb(230, 230, 230)",
            gridcolor="white",
            showbackground=True,
            title="X"
        ),
        yaxis=dict(
            backgroundcolor="rgb(230, 230, 230)",
            gridcolor="white",
            showbackground=True,
            title="Y"
        ),
        zaxis=dict(
            backgroundcolor="rgb(230, 230, 230)",
            gridcolor="white",
            showbackground=True,
            title="Z"
        )
    ),
    width=900,
    height=700,
    title="医学图像 3D 重建",
    paper_bgcolor='white',
    plot_bgcolor='white'
)

fig.show()